In [2]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import pymysql
from collections import Counter
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
conn = pymysql.connect(host='chamaeleon.konkuk.ac.kr',
                       port = 3306,
                       user = "class_da",
                       password = "dataanalytics",
                       db = "tp_class_da")
cursor = conn.cursor()
cursor.execute("SELECT ip.특허등록번호 as pt_num, ip.보유IPC전체코드 as ipc_code FROM ipc ip")
db = cursor.fetchall()

In [4]:
ipc_list = []

for pt_num, ipc_code in db:
    ipc_list.append((pt_num, ipc_code[0]))

ipc_dict = {}

for num, ipc_code in ipc_list:
    if ipc_dict.get(num):
        ipc_dict[num].append(ipc_code)
    else :
        ipc_dict[num]=[ipc_code]

In [5]:
ipc_dict = {key:value for key, value in ipc_dict.items() if len(Counter(ipc_dict[key])) != 1}

In [6]:
ipc_dict
len(ipc_dict)

120065

In [6]:
pt_num_list = list(ipc_dict.keys())

col = ["A", "B", "C", "D", "E", "F", "G", "H"]

df = DataFrame(columns = col, index = pt_num_list)
df_num = DataFrame(columns = col, index = pt_num_list)

for i in pt_num_list:
    for j in col:
        df_num.at[i, j] = Counter(ipc_dict[i])[j]

        
for i in pt_num_list:
    for j in col:
        if Counter(ipc_dict[i])[j] > 0:
            df.at[i, j] = True
        else:
            df.at[i, j] = False

In [7]:
co_df = np.dot(df_num.T, df_num)
co_df

array([[556794, 58055, 203850, 3901, 5009, 16666, 82066, 29600],
       [58055, 646557, 133429, 10238, 24625, 74863, 83735, 75719],
       [203850, 133429, 783565, 8273, 7357, 15942, 36258, 62641],
       [3901, 10238, 8273, 27590, 388, 1184, 1022, 1462],
       [5009, 24625, 7357, 388, 62475, 12227, 10588, 4876],
       [16666, 74863, 15942, 1184, 12227, 253360, 28552, 38356],
       [82066, 83735, 36258, 1022, 10588, 28552, 719974, 349318],
       [29600, 75719, 62641, 1462, 4876, 38356, 349318, 976886]],
      dtype=object)

In [8]:
df

,A,B,C,D,E,F,G,H
9532498,True,False,False,False,False,True,False,False
9532499,True,False,False,False,False,False,True,False
9532503,True,False,False,False,False,True,False,False
9532507,True,True,False,False,False,False,False,False
9532508,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...
9854712,False,False,False,False,False,True,True,True
9854714,False,False,False,False,False,True,False,True
9854715,False,False,False,False,False,True,False,True
9854716,False,False,False,False,False,False,True,True


In [44]:
frequent_itemsets=apriori(df, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[:20]

,support,itemsets,length
0,0.248507,(A),1
1,0.3388,(B),1
2,0.241136,(C),1
3,0.0564861,(E),1
4,0.167834,(F),1
5,0.578503,(G),1
6,0.509791,(H),1
7,0.0611336,"(A, B)",2
8,0.0983467,"(C, A)",2
9,0.085004,"(A, G)",2


In [46]:
rules=association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
rules[:10]
rules.sort_values(by=['support'], ascending=False)[:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
14,(H),(G),0.509791,0.578503,0.385158,0.755522,1.305995,0.090243,1.724069
15,(G),(H),0.578503,0.509791,0.385158,0.665784,1.305995,0.090243,1.466743
10,(G),(B),0.578503,0.338800,0.104668,0.180929,0.534031,-0.091329,0.807257
11,(B),(G),0.338800,0.578503,0.104668,0.308938,0.534031,-0.091329,0.609926
2,(C),(A),0.241136,0.248507,0.098347,0.407847,1.641191,0.038423,1.269087
3,(A),(C),0.248507,0.241136,0.098347,0.395750,1.641191,0.038423,1.255878
6,(C),(B),0.241136,0.338800,0.085379,0.354069,1.045068,0.003682,1.023639
7,(B),(C),0.338800,0.241136,0.085379,0.252004,1.045068,0.003682,1.014529
4,(A),(G),0.248507,0.578503,0.085004,0.342059,0.591282,-0.058758,0.640629
5,(G),(A),0.578503,0.248507,0.085004,0.146938,0.591282,-0.058758,0.880936
